In [1]:
!pip install -qU \
    datasets==2.12.0 \
    apache_beam \
    mwparserfromhell

^C


In [6]:
with open('meditations copy 2.text', "r") as f: 
    data = f.read()

len(data)

314190

In [8]:
!pip install langchain==0.0.162 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  "pinecone-client[grpc]"==2.2.2

  Using cached langchain-0.0.162-py3-none-any.whl (770 kB)
  Using cached openai-0.27.7-py3-none-any.whl.metadata (13 kB)
  Using cached tiktoken-0.4.0-cp310-cp310-macosx_10_9_x86_64.whl (797 kB)
  Using cached pinecone_client-2.2.2-py3-none-any.whl.metadata (7.8 kB)
  Using cached SQLAlchemy-2.0.25-cp310-cp310-macosx_10_9_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.1-cp310-cp310-macosx_10_9_x86_64.whl.metadata (7.4 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl.metadata (22 kB)
  Using cached numexpr-2.8.8-cp310-cp310-macosx_10_9_x86_64.whl.metadata (7.9 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached pydantic-1.10.13-cp310-cp310-macosx_10_9_x86_64.whl.metadata (149 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached regex-2023.12.25-cp310-cp310-macosx_10_9_x86_64.whl.metadata (40 kB)
  Using cached grpcio-1.60.0.tar.gz

In [96]:
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def split_into_sentence_chunks(text, max_chunk_length):
    sentences = sent_tokenize(text)
    
    current_chunk = ""
    chunks = []

    for sentence in sentences:
        if len(current_chunk) + len(sentence) > max_chunk_length:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

file_contents = data
max_chunk_length = 200  # Choose the maximum length for each chunk
sentence_chunks = split_into_sentence_chunks(file_contents, max_chunk_length)


In [97]:
sentence_chunks = [sentence for sentence in sentence_chunks if len(sentence) >= 10]

In [98]:
sentence_chunks[0]

'The Roman Emperor His First Book oncerning himself Wherein Antoninus recordeth, What and of whom, whether Parents, Friends, or asters; by their good examples, or good advice and counsel, he had learned: ivided into Numbers or Sections.'

In [99]:
import os

# get openai api key from platform.openai.com
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

In [100]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [101]:
res = embed.embed_documents(sentence_chunks)
len(res), len(res[0])

(1558, 1536)

In [149]:
index_name = 'pdfsearch'

In [150]:
import pinecone

# find API key in console at app.pinecone.io
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# find ENV (cloud region) next to API key in console
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key='PINECONE_API_KEY',
    environment='PINECONE_ENVIRONMENT'
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=len(res[0])  # 1536 dim of text-embedding-ada-002
    )

In [151]:
index = pinecone.GRPCIndex(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [152]:
from tqdm.auto import tqdm
from uuid import uuid4
import time

batch_size = 250
for i in tqdm(range(0, len(sentence_chunks), batch_size)):
    
    i_min = min(i+batch_size, len(sentence_chunks))
    batch = sentence_chunks[i: i_min]
    meta_data = [{"titile" : 'Marcus Aurelius Meditations', 
              "context": row}
                for row in batch]
    ids = [str(uuid4()) for _ in range(len(batch))]
    # Encode the text to obtain its vector representation
    embeds = embed.embed_documents(batch)
    
    # Upsert the vector and text into the Pinecone index
    index.upsert(vectors=zip(ids, embeds, meta_data))
    time.sleep(4)



 71%|████████████████████████████████▏            | 5/7 [00:52<00:21, 10.78s/it]Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DALsQsfLBMuaNvlMG6TH8Z7n on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DALsQsfLBMuaNvlMG6TH8Z7n on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method t

In [153]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.01558,
 'namespaces': {'': {'vector_count': 1558}},
 'total_vector_count': 1558}

In [154]:
from langchain.vectorstores import Pinecone

text_field = "context"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [170]:
query = "Explore the dichotomy between the inner self and external circumstances as presented by Marcus Aurelius"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='then thou shalt separate from thyself, that is from thy mind, whatsoever other men either do or say, or whatsoever thou thyself hast heretofore either done or said; and all trouble- some thoughts concerning the future, and whatsoever, (as either belonging to thy body or life): is without the jurisdiction of thine own will, and whatsoever in the ordi- nary course of human chances and accidents doth happen unto thee; so that thy mind (keeping herself loose and free from all outward coincidental entanglements; always in a readiness to depart): shall live by herself, and to herself, doing that which is just, accepting whatsoever doth happen, and speaking the truth always; if, say, thou shalt separate from thy mind, whatsoever by sympathy might adhere unto it, and all time both past and future, and shalt make thyself in all points and respects, like unto Empedocles his allegorical sphere, “all round and circular,” &., and shalt think of no longer life than that which

In [157]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory \
import ConversationBufferWindowMemory

from langchain.chains import RetrievalQA

# OpenAI LLM
llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY,
                model_name = 'gpt-3.5-turbo',
                temperature = 0.0)

# conversational memory
conv_mem = ConversationBufferWindowMemory(
    memory_key = 'chat_history',
    k = 5,
    return_messages =True)

# retrieval qa
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore.as_retriever())

In [171]:
query = "Explore the dichotomy between the inner self and external circumstances as presented by Marcus Aurelius"
qa.run(query)

"In the passage, Marcus Aurelius emphasizes the importance of separating oneself from external circumstances and focusing on the inner self. He advises to detach from the thoughts and actions of others, as well as from past and future events that are beyond one's control. By doing so, one can live with a free and clear mind, accepting whatever happens and speaking the truth.\n\nAurelius suggests that true philosophy lies in preserving one's inner spirit, regardless of insults, injuries, or even pleasures and pains. It involves acting sincerely and authentically, relying on one's own actions, and embracing everything that comes as part of a greater plan. He encourages a calm acceptance of death, viewing it as the natural dissolution of the elements that make up every living being.\n\nThe passage also highlights the power of one's own thoughts and opinions in shaping their actions and emotions. Aurelius states that it is not external circumstances that influence a person, but rather thei